방문 주차에 따른 코호트를 생성하고, 코호트 별로 weekly 리텐션을 구하고 이를 heatmap으로 시각화

In [ ]:
# week 칼럼 만들기 -> 비교하는 칼럼 만들기
# df['week'] = df['visitStartTime'].dt.week
# month_df = df.groupby('period_month')[['fullVisitorId']].nunique()
# retention = 지난주 접속자 수 / 이번주 접속자 수
# 어렵네요..
# n = len(wau_df) # 53

In [ ]:
# plt.rcParams['figure.figsize'] = [10,8]
# plt.pcolor(table)
# # 또는 sns.heatmap()
# # 피벗테이블의 df를 pandas를 이용해 표현도 가능 df.style.background_gradient
# plt.title('Weekly Retention', fontsize=20)
# plt.xlabel('week', fontsize=14)
# plt.ylabel('visit', fontsize=14)
# plt.colorbar()

In [ ]:
# 전처리한 데이터 불러오기
import pandas as pd

df = pd.read_csv('C://Users//and_c//Desktop//sparta_ga4//train_df3.csv', low_memory = False)

In [ ]:
df.info

In [ ]:
df.head()

In [ ]:
# 데이터프레임에 한 열의 값을 바꾸기 위해서 def함수를 만들고 apply 적용
import pandas as pd
import datetime #날짜와 시간을 다루는 라이브러리
import pytz #다양한 시간대를 처리하기 위해 사용하는 라이브러리

# Posix 시간을 utc 시간으로 변환하는 함수
def posix_to_utc(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime)
    return utc_time

# Posix 시간을 UTC 지정한 로컬 시간대로 변환하는 함수
def posix_to_utc_local(visitStartTime):
    utc_time = datetime.datetime.utcfromtimestamp(visitStartTime) #posix -> utc시간대로
    local_timezone = pytz.timezone('America/New_York')  # 지역 시간대
    return utc_time.astimezone(local_timezone) # 그 지역 시간대로 변환

In [ ]:
df['visitStartTime'] = df['visitStartTime'].apply(posix_to_utc)

In [ ]:
# date 칼럼 추가하기
import datetime as dt

def get_date(x):
    return dt.datetime(x.year, x.month, x., 1)

df['date'] = df['visitStartTime'].apply(get_date)
